In [ ]:
pip install requests

In [ ]:
pip install progress_bar

In [ ]:
pip install selenium

In [ ]:
pip install beautifulsoup4

In [1]:
# import necessary modules
import time
import datetime
import random
import numpy as np
import base64

import re
import os

import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

from tqdm import tqdm
import urllib.request 
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.expected_conditions import presence_of_element_located
# browser.quit() # close the whole browser
# browser.close() # close the tab

In [2]:
def get_product_info_from_mainpage(URL):
    product_links = []
    image_links = []
    prices = []
    
    driver = webdriver.Chrome('C:/Users/Brian/Downloads/chromedriver_win32/chromedriver.exe')
    driver.implicitly_wait(100)
    
    try:
        driver.get(URL)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        time.sleep(10)
        
        product_cards = soup.find_all('div', {"class" : "ProductCard-container ProductCard-container--contained"})

        print("Number of products scraped from the page: ", len(product_cards))
        
        i = 0
        for product_card in product_cards:
            try:
                #get image link
                driver.implicitly_wait(3)
                image = product_card.find('div', {"class" : "BrowseProductCard-imageWrap"})
                #image_link = image.find_element_by_tag_name('img').get_attribute('src')
                #image_link = image.find('img')["src"]
                #image_link = image.select('img').get('src')
                image_link = image.find('img')["srcset"].split(',')[-1].split(" ")[0]
                image_links.append(image_link)

                
                #get product link
                product_link = product_card.find('a')['href']
                product_links.append(product_link)                


                #get price
                price_block = product_card.find('div', {"class" : "ProductCard-pricing"})
                price = price_block.find('span').text

                '''
                try:
                    price = price_block.find('span', {"class" : "ProductCard-price"}).text
                except:
                    price = price_block.find('span', {"class" : "ProductCard-price ProductCard-price--sale"}).text
                '''
                
                prices.append(price)
                i += 1
                
                #Print summary information
                print(f"Item: {i}\n")
                print(f"Product link: {product_link}\n")
                print(f"Image link: {image_link}\n")
                print(f"Price: {price}\n")
                print("==============================================")
            
            except:
                print("Error")
                pass
        driver.close()
        print("Lengths of lists: ", len(product_links), len(image_links), len(prices))
        return (product_links, image_links, prices)
    
    except TimeoutException as e:
        print("Page load Timeout Occured. Quitting !!!")
        driver.close()

'''
def get_info_from_productlink(product_link):  #get the first image
    info = dict()
    page = requests.get(product_link)
    driver = webdriver.Chrome('C:/Users/Brian/Downloads/chromedriver_win32/chromedriver.exe')
    driver.get(product_link)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    time.sleep(random.randint(3,3))

    #price_block = soup.find('div', {"class" : "BasePriceBlock BasePriceBlock--highlight"})
    #price_block = soup.find('div', {"class" : "BasePriceBlock"})
    price_block = soup.find('span', {"class" : "notranslate"})
    price = price_block.text
    info['price'] = price
    #print(price)   
    
    image = soup.find('div', {'class' : "ImageComponent ImageComponent--overlay"})
    image_link = image.find('img')['src']
    info['img_url'] = image_link
    
    #print(f"Image link: {image_link}\n")
    #print(f"Link to product: {product_link}\n")

    driver.close()
    #browser.switch_to.window(browser.window_handles[0])
    return info
'''

'\ndef get_info_from_productlink(product_link):  #get the first image\n    info = dict()\n    page = requests.get(product_link)\n    driver = webdriver.Chrome(\'C:/Users/Brian/Downloads/chromedriver_win32/chromedriver.exe\')\n    driver.get(product_link)\n    soup = BeautifulSoup(driver.page_source, \'html.parser\')\n    time.sleep(random.randint(3,3))\n\n    #price_block = soup.find(\'div\', {"class" : "BasePriceBlock BasePriceBlock--highlight"})\n    #price_block = soup.find(\'div\', {"class" : "BasePriceBlock"})\n    price_block = soup.find(\'span\', {"class" : "notranslate"})\n    price = price_block.text\n    info[\'price\'] = price\n    #print(price)   \n    \n    image = soup.find(\'div\', {\'class\' : "ImageComponent ImageComponent--overlay"})\n    image_link = image.find(\'img\')[\'src\']\n    info[\'img_url\'] = image_link\n    \n    #print(f"Image link: {image_link}\n")\n    #print(f"Link to product: {product_link}\n")\n\n    driver.close()\n    #browser.switch_to.window(bro

In [ ]:
#TEST
get_product_info_from_mainpage('https://www.wayfair.com/filters/furniture/sb1/sofas-c413892-p2479~1000~*.html?itemsperpage=96&curpage=2')

In [20]:
def save_data(product_info_list, i):
    result = [[product[0], product[1]['price'], product[1]['img_url']] for product in product_info_list]
    df = pd.DataFrame(result)
    now = str(datetime.datetime.now()).replace(" ","").replace(":","").replace("-","").split(".")[0][4:][:-2]
    df.to_csv(f'sofa_info_{str(i)}.csv', index=False, header=['item_no', 'price', 'img_url'])
    
def save_image_from_url(product, category):    
    bar_wrap = [None]

    item_no = product.name + 5780
    print("Product Row: ", product.name)
    base_name = "C:/Users/Brian/Desktop/ThePriceIsRight/" + category + "/"
    filename = f"./{category}/{item_no}.jpeg"
    filepath = base_name + str(item_no) + ".jpg"
    url = str(product['img_url'])
    
    try:
        print(url)
        urllib.request.urlretrieve(url, filepath)
        return filename
    except:
        print(f"Failed to download image:{url}\n")
        return f"Failed to download image: {url}\n"
    
def save_images_from_csv(df):
    tqdm.pandas()
    df['filename'] = df.progress_apply(lambda x: save_image_from_url(x, category = 'sofa'), axis=1)
    return df

In [4]:
product_info = []
base_url = 'https://www.wayfair.com/filters/furniture/sb1/sofas-c413892-p2479~1000~*.html?itemsperpage=96&curpage='
#base_url = 'https://www.wayfair.com/furniture/sb0/sofas-c413892.html?curpage='

#data from pages 1 - 4
for page in tqdm(range(1,5)):
    url = base_url + str(page)
    (product_links, image_links, prices) = get_product_info_from_mainpage(url)
    print(f"Number of products on Page {page}: {len(product_links)}\n")
    i = 5779
    for (product_url, img_url, price) in tqdm(zip(product_links, image_links, prices)):
        i += 1
        #item_no = f"{page}_{i}"
        item_no = f"{i}"
        try:
            info = {'price' : price, 'img_url' : img_url}
            product_info.append((item_no,info))     
        except:
            print(f"Failed. Product link: {product_url}")
            pass

  0%|                                                                                            | 0/4 [00:00<?, ?it/s]

Number of products scraped from the page:  96
Item: 1

Product link: https://www.wayfair.com/furniture/pdp/house-of-hampton-chesterfield-845-rolled-arm-sofa-w001297038.html?piid=1634155177

Image link: https://secure.img1-fg.wfcdn.com/im/12726311/resize-h310-w310%5Ecompr-r85/6493/64936323/Chesterfield+84.5%2522+Rolled+Arm+Sofa.jpg

Price: $1,019.99

Item: 2

Product link: https://www.wayfair.com/furniture/pdp/everly-quinn-luanna-chesterfield-80-recessed-arm-sofa-eyqn6074.html?piid=25359506%2C50099778&categoryid=413892&placement=1&slot=0&sponsoredid=b86f0c0a33e69d53e6c4100df1d3185ec016ee0c3460ca6b3b81d5cdfb52ed7b&_txid=I%2BF9Ol8WkeQqaA%2F5srh8Ag%3D%3D&isB2b=0&auctionId=3fabb403-8ca5-4e98-af22-fd99230aa2a3

Image link: https://secure.img1-fg.wfcdn.com/im/56432728/resize-h310-w310%5Ecompr-r85/1178/117801067/Luanna+Chesterfield+80%2522+Recessed+Arm+Sofa.jpg

Price: $1,019.99

Item: 3

Product link: https://www.wayfair.com/furniture/pdp/trent-austin-design-bombay-genuine-leather-84-square-a


Price: $1,069.99

Item: 22

Product link: https://www.wayfair.com/furniture/pdp/orren-ellis-alisia-323-armless-sofa-w000409763.html?piid=279767931&categoryid=413892&placement=1&slot=4&sponsoredid=548d2b5da8485561a3faee9826dbbfc8b6fdb8fc8e6f26e8b6a00fce15ec1808&_txid=I%2BF9Ol8WkeQqaA%2F5srh8Ag%3D%3D&isB2b=0&auctionId=3fabb403-8ca5-4e98-af22-fd99230aa2a3

Image link: https://secure.img1-fg.wfcdn.com/im/72375181/resize-h310-w310%5Ecompr-r85/1054/105463016/Alisia+32.3%2522+Armless+Sofa.jpg

Price: $2,059.99

Item: 23

Product link: https://www.wayfair.com/furniture/pdp/astoria-grand-welles-loveseat-w000012189.html?piid=745158252

Image link: https://secure.img1-fg.wfcdn.com/im/25649198/resize-h310-w310%5Ecompr-r85/5877/58774200/Welles+Loveseat.jpg

Price: $2,699.99

Item: 24

Product link: https://www.wayfair.com/furniture/pdp/darby-home-co-kavya-chesterfield-109-tuxedo-arm-sofa-drbh4573.html?piid=25729255

Image link: https://secure.img1-fg.wfcdn.com/im/10577193/resize-h310-w310%5Ecompr-


Price: $1,669.99

Item: 45

Product link: https://www.wayfair.com/furniture/pdp/orren-ellis-ketan-right-hand-facing-sectional-orne3077.html?piid=23545000

Image link: https://secure.img1-fg.wfcdn.com/im/59931372/resize-h310-w310%5Ecompr-r85/3872/38721378/Ketan+Right+Hand+Facing+Sectional.jpg

Price: $1,339.99

Item: 46

Product link: https://www.wayfair.com/furniture/pdp/magnolia-home-3775-square-arm-loveseat-mgnl1331.html?piid=&categoryid=413892&placement=1&slot=6&sponsoredid=4127590bfe962cec3fc02cd4a2120750fb6528905d58f9c9c528d6f788ff0ec1&_txid=I%2BF9Ol8WkeQqaA%2F5srh8Ag%3D%3D&isB2b=0&auctionId=3fabb403-8ca5-4e98-af22-fd99230aa2a3

Image link: https://secure.img1-fg.wfcdn.com/im/96840003/resize-h310-w310%5Ecompr-r85/1181/118101839/37.75%2527%2527+Square+Arm+Loveseat.jpg

Price: $1,189.99

Item: 47

Product link: https://www.wayfair.com/furniture/pdp/millwood-pines-charlie-sierra-sofa-w000558557.html?piid=

Image link: https://secure.img1-fg.wfcdn.com/im/83441234/resize-h310-w310%5Ec


Image link: https://secure.img1-fg.wfcdn.com/im/43342383/resize-h310-w310%5Ecompr-r85/4128/41288863/Lambdin+Genuine+Leather+88%2522+Round+Arm+Sofa.jpg

Price: $1,389.99

Item: 67

Product link: https://www.wayfair.com/furniture/pdp/three-posts-whipton-84-inches-rolled-arms-sofa-w001671155.html?piid=

Image link: https://secure.img1-fg.wfcdn.com/im/97702415/resize-h310-w310%5Ecompr-r85/8947/89477375/Whipton+84+inches+Rolled+Arms+Sofa.jpg

Price: $1,199.99

Item: 68

Product link: https://www.wayfair.com/furniture/pdp/world-menagerie-destanee-94-rolled-arm-sofa-wldm7988.html?piid=

Image link: https://secure.img1-fg.wfcdn.com/im/21917770/resize-h310-w310%5Ecompr-r85/7592/75924787/Destanee+94%2522+Rolled+Arm+Sofa.jpg

Price: $1,229.99

Item: 69

Product link: https://www.wayfair.com/furniture/pdp/world-menagerie-hotchkiss-genuine-leather-84-rolled-arm-sofa-w002509746.html?piid=

Image link: https://secure.img1-fg.wfcdn.com/im/50321908/resize-h310-w310%5Ecompr-r85/7892/78928988/Hotchkiss+

...

Number of products scraped from the page:  96
Item: 1

Product link: https://www.wayfair.com/furniture/pdp/house-of-hampton-koffler-velvet-chesterfield-94-rolled-arm-sofa-w003012356.html?piid=1180209226

Image link: https://secure.img1-fg.wfcdn.com/im/10401226/resize-h310-w310%5Ecompr-r85/4845/48455940/Koffler+Velvet+Chesterfield+94%2522+Rolled+Arm+Sofa.jpg

Price: $1,119.99

Item: 2

Product link: https://www.wayfair.com/furniture/pdp/canora-grey-barkell-leather-91-rolled-arms-sofa-cagy1721.html?piid=

Image link: https://secure.img1-fg.wfcdn.com/im/22515703/resize-h310-w310%5Ecompr-r85/8636/86368296/Barkell+Leather+91%2522+Rolled+Arms+Sofa.jpg

Price: $2,799.99

Item: 3

Product link: https://www.wayfair.com/furniture/pdp/orren-ellis-eisley-loveseat-w001516846.html?piid=

Image link: https://secure.img1-fg.wfcdn.com/im/94701072/resize-h310-w310%5Ecompr-r85/1041/104143420/Eisley+Loveseat.jpg

Price: $1,639.99

Item: 4

Product link: https://www.wayfair.com/furniture/pdp/orren-ellis-ne


Price: $1,429.99

Item: 26

Product link: https://www.wayfair.com/furniture/pdp/red-barrel-studio-wilpert-reclining-86-pillow-top-arm-sofa-w003208518.html?piid=954173167

Image link: https://secure.img1-fg.wfcdn.com/im/42981816/resize-h310-w310%5Ecompr-r85/1189/118959884/Wilpert+Reclining+86%2527%2527+Pillow+Top+Arm+Sofa.jpg

Price: $3,099.99

Item: 27

Product link: https://www.wayfair.com/furniture/pdp/canora-grey-barkell-leather-70-rolled-arms-loveseat-cagy1722.html?piid=

Image link: https://secure.img1-fg.wfcdn.com/im/55223511/resize-h310-w310%5Ecompr-r85/4149/41498152/Barkell+Leather+70%2522+Rolled+Arms+Loveseat.jpg

Price: $2,179.99

Item: 28

Product link: https://www.wayfair.com/furniture/pdp/corrigan-studio-calvo-sofa-w000976085.html?piid=528145181

Image link: https://secure.img1-fg.wfcdn.com/im/57611497/resize-h310-w310%5Ecompr-r85/1035/103594293/Calvo+Sofa.jpg

Price: $1,219.99

Item: 29

Product link: https://www.wayfair.com/furniture/pdp/dreamseat-tottenham-hotspur-prim


Price: $1,799.99

Item: 50

Product link: https://www.wayfair.com/furniture/pdp/17-stories-chien-sofa-stss8851.html?piid=

Image link: https://secure.img1-fg.wfcdn.com/im/35955891/resize-h310-w310%5Ecompr-r85/1179/117950133/Chien+Sofa.jpg

Price: $2,659.99

Item: 51

Product link: https://www.wayfair.com/furniture/pdp/rosdorf-park-luther-chesterfield-66-rolled-arm-loveseat-w001566911.html?piid=444293603

Image link: https://secure.img1-fg.wfcdn.com/im/10010104/resize-h310-w310%5Ecompr-r85/7065/70659107/Luther+Chesterfield+66%2522+Rolled+Arm+Loveseat.jpg

Price: $1,229.99

Item: 52

Product link: https://www.wayfair.com/furniture/pdp/corrigan-studio-amaro-84-square-arm-sofa-w003085314.html?piid=910623968

Image link: https://secure.img1-fg.wfcdn.com/im/38280247/resize-h310-w310%5Ecompr-r85/1141/114140788/Amaro+84%2522+Square+Arm+Sofa.jpg

Price: $2,499.99

Item: 53

Product link: https://www.wayfair.com/furniture/pdp/ebern-designs-stanley-chesterfield-sofa-w001977623.html?piid=14094450


Image link: https://secure.img1-fg.wfcdn.com/im/67386363/resize-h310-w310%5Ecompr-r85/4568/45680799/Theodora+Genuine+Leather+89%2522+Square+Arm+Sofa.jpg

Price: $1,439.99

Item: 75

Product link: https://www.wayfair.com/furniture/pdp/orren-ellis-francis-brown-leather-sofa-orel2898.html?piid=

Image link: https://secure.img1-fg.wfcdn.com/im/92707293/resize-h310-w310%5Ecompr-r85/2678/26783769/Francis+Brown+Leather+Sofa.jpg

Price: $3,599.99

Item: 76

Product link: https://www.wayfair.com/furniture/pdp/lexington-avondale-60-flared-arm-settee-ltn10450.html?piid=

Image link: https://secure.img1-fg.wfcdn.com/im/68095848/resize-h310-w310%5Ecompr-r85/1184/118451678/Avondale+60%2522+Flared+Arm+Settee.jpg

Price: $2,129.00

Item: 77

Product link: https://www.wayfair.com/furniture/pdp/orren-ellis-cowhill-leather-loveseat-ornl3545.html?piid=25936360

Image link: https://secure.img1-fg.wfcdn.com/im/15335895/resize-h310-w310%5Ecompr-r85/4774/47748286/Cowhill+Leather+Loveseat.jpg

Price: $2,599.9


92it [00:00, 92248.62it/s]
100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [01:22<00:00, 20.61s/it]


In [21]:
save_data(product_info,1)
product_info

[('5780',
  {'price': '$1,019.99',
   'img_url': 'https://secure.img1-fg.wfcdn.com/im/12726311/resize-h310-w310%5Ecompr-r85/6493/64936323/Chesterfield+84.5%2522+Rolled+Arm+Sofa.jpg'}),
 ('5781',
  {'price': '$1,019.99',
   'img_url': 'https://secure.img1-fg.wfcdn.com/im/56432728/resize-h310-w310%5Ecompr-r85/1178/117801067/Luanna+Chesterfield+80%2522+Recessed+Arm+Sofa.jpg'}),
 ('5782',
  {'price': '$1,079.99',
   'img_url': 'https://secure.img1-fg.wfcdn.com/im/38238360/resize-h310-w310%5Ecompr-r85/6045/60454011/Bombay+Genuine+Leather+84%2522+Square+Arms+Sofa.jpg'}),
 ('5783',
  {'price': '$1,169.99',
   'img_url': 'https://secure.img1-fg.wfcdn.com/im/70316309/resize-h310-w310%5Ecompr-r85/4687/46877341/Hardin+Right+Hand+Facing+Sectional+with+Ottoman.jpg'}),
 ('5784',
  {'price': '$1,119.99',
   'img_url': 'https://secure.img1-fg.wfcdn.com/im/16981263/resize-h310-w310%5Ecompr-r85/1047/104796711/Westberg+Velvet+Chesterfield+76%2522+Rolled+Arms+Loveseat.jpg'}),
 ('5785',
  {'price': '$1,349

In [22]:
df = pd.read_csv('sofa_info_1.csv')
print(df.shape)
df.reset_index(drop=True, inplace=True)

(368, 3)


In [23]:
df

,item_no,price,img_url
0,5780,"$1,019.99",https://secure.img1-fg.wfcdn.com/im/12726311/r...
1,5781,"$1,019.99",https://secure.img1-fg.wfcdn.com/im/56432728/r...
2,5782,"$1,079.99",https://secure.img1-fg.wfcdn.com/im/38238360/r...
3,5783,"$1,169.99",https://secure.img1-fg.wfcdn.com/im/70316309/r...
4,5784,"$1,119.99",https://secure.img1-fg.wfcdn.com/im/16981263/r...
...,...,...,...
363,5867,"$1,239.99",https://secure.img1-fg.wfcdn.com/im/52130521/r...
364,5868,"$1,159.99",https://secure.img1-fg.wfcdn.com/im/08804911/r...
365,5869,"$3,559.99",https://secure.img1-fg.wfcdn.com/im/83273983/r...
366,5870,"$1,049.99",https://secure.img1-fg.wfcdn.com/im/98654611/r...


In [24]:
df_new = save_images_from_csv(df)
df_new.to_csv('sofa.csv', index=False)

  0%|                                                                                          | 0/368 [00:00<?, ?it/s]

Product Row:  0
https://secure.img1-fg.wfcdn.com/im/12726311/resize-h310-w310%5Ecompr-r85/6493/64936323/Chesterfield+84.5%2522+Rolled+Arm+Sofa.jpg


  1%|▍                                                                                 | 2/368 [00:00<00:29, 12.53it/s]

Product Row:  1
https://secure.img1-fg.wfcdn.com/im/56432728/resize-h310-w310%5Ecompr-r85/1178/117801067/Luanna+Chesterfield+80%2522+Recessed+Arm+Sofa.jpg


  1%|▋                                                                                 | 3/368 [00:00<00:38,  9.58it/s]

...

100%|███████████████████████████████████████████████████████████████████████████████▊| 367/368 [00:55<00:00,  7.01it/s]

Product Row:  366
https://secure.img1-fg.wfcdn.com/im/98654611/resize-h310-w310%5Ecompr-r85/1166/116624039/Passerine+Cotton+58%2522+Recessed+Arms+Loveseat.jpg


100%|████████████████████████████████████████████████████████████████████████████████| 368/368 [00:56<00:00,  6.83it/s]

Product Row:  367
https://secure.img1-fg.wfcdn.com/im/90060031/resize-h310-w310%5Ecompr-r85/6859/68597719/Bloomsburg+Leather+Loveseat.jpg


100%|████████████████████████████████████████████████████████████████████████████████| 368/368 [00:56<00:00,  6.55it/s]


In [25]:
df_new

,item_no,price,img_url,filename
0,5780,"$1,019.99",https://secure.img1-fg.wfcdn.com/im/12726311/r...,./sofa/5780.jpeg
1,5781,"$1,019.99",https://secure.img1-fg.wfcdn.com/im/56432728/r...,./sofa/5781.jpeg
2,5782,"$1,079.99",https://secure.img1-fg.wfcdn.com/im/38238360/r...,./sofa/5782.jpeg
3,5783,"$1,169.99",https://secure.img1-fg.wfcdn.com/im/70316309/r...,./sofa/5783.jpeg
4,5784,"$1,119.99",https://secure.img1-fg.wfcdn.com/im/16981263/r...,./sofa/5784.jpeg
...,...,...,...,...
363,5867,"$1,239.99",https://secure.img1-fg.wfcdn.com/im/52130521/r...,./sofa/6143.jpeg
364,5868,"$1,159.99",https://secure.img1-fg.wfcdn.com/im/08804911/r...,./sofa/6144.jpeg
365,5869,"$3,559.99",https://secure.img1-fg.wfcdn.com/im/83273983/r...,./sofa/6145.jpeg
366,5870,"$1,049.99",https://secure.img1-fg.wfcdn.com/im/98654611/r...,./sofa/6146.jpeg
